# Protocol for a distributed synthetic market

This is a protocol for matching **state contingent contracts** that give a payoff linear between an upper and lower bound and at a particular expiry.

<img src="img/BFC.png"  width=500>

For example the contract might pay \$1 for each 10mm of rainfall above 30mm. If the amount of rainfall is above 30mm the payoff is \$1 and if the outcome is below 30mm the outcome is zero.

These contracts can be combined to construct standard forward and option contracts or event markets (payoffs depending on a discrete set of outcomes). 

<img src="img/StackedBFC.png" width=500>

The protocol uses a chain of signatures to organize buy and sell orders for these contracts. Orders are accepted if they have a valid signature and the trader's positions satisfiy a collateral check across all possible worst outcomes. To maintain an appropriate record of the order book, trades can only be added, and the validity of any trade can be checked by anyone using public keys provided by each participant.

Markets are settled by the market owner adding a new upper and lower bound for a market equal to a single number. For example, the lower bound for each market is the highest lower bound for itself of  any sub-market or any previous bound on the same market. Market bounds *telescope*, making a market the *underlying asset* of its sub-markets. 

<img src="img/marketBounds.png" width=600>





## Data structures

### A market

A **market** is defined by:

- 'marketRootId': (integer) market id
- 'marketBranchId': (integer) sub-markets >1 (sub-markets bounded by super-markets)
- 'marketMin' - (float) minimum possible outcome
- 'marketMax' - (float) maximum possible outcome
- 'traderId' - (integer) market owner trader ID
- 'signatureMsg' - (string) message for signature
- 'signature' - (binary) signed message

E.g.

Market with (root = 3, branch = 1); market bounded between (0, 1); owned by trader 1 and signed with signature 'sig1'.

~~~~

testMarket = struct('marketRootId', 3, 'marketBranchId', 1,...
                                'marketMin', 0, 'marketMax', 1,...
                                'traderId', 1, 'signatureMsg','sigmsg1',...
                                'signature', 'sig1')

~~~~

- Sub markets have marketMin/marketMax bounded by markets with the same root but lower branch id.
- Any amount of valid markets with the same root/branch can be added and the market will take the highest minimum and the lowest maximum.





### An order

An **order** in a market is defined by:

 - 'tradeRootId': (integer) trade root id
 - 'tradeBranchId': (integer) subtrades (dependent trades such as offsets and partials)
 - 'isMatched': (boolean) is trade matched (adding an unmatched trade requries adding an offsetting trade and adding an equal sized matched trade from cache)
 - 'price':  (float) price of trade
 - 'quantity':  (float) quanitity of trade (positive quantity for bids, negative for offers) 
 - 'marketRootId' : (integer) market id
 - 'marketBranchId': (integer) sub-markets > 1 (sub-markets bounded by super-markets)
 - 'traderId':  (integer) trade owner trader ID
 - 'signatureMsg' - (string) message for signature
 - 'signature' - (binary) signed message

E.g. 

Unmatched primary trade (price = 0.5, quantity = 10) on market (root = 1, branch = 1)

~~~~

testTrade = struct('traderId', 1, 'tradeRootId', 1, 'tradeBranchId', 1,...
                'isMatched', 0, 'price', 0.5,...
                'quantity', 10, 'marketRootId', 3,...
                'marketBranchId', 1,'signatureMsg',...
                'sigMsg', 'signature', 'sig');

~~~~


Trades can be added to the order book but not removed or changed. A separate cache order book is maintained for partial trades, offsets, and matches. The cache order book is ignored for collateral calculations.





#### Geometry of an order

Since orders can only be added to the order book and not subtracted, creating a matched order requires an offset of the unmatched order and a new matched order of equal size.

Consider an order '(p=0.5, q=1)'. For the trade to successfully be matched it requires at minimum:

- Primary  (p=0.5, q=1, isMatched = 0) 
- Offset  (p=0.5, q=-1, isMatched = 0)
- Match (p=0.5, q=1, isMatchced = 1)


Geometrically, this set of trades can be represented in (p,q) space with with offset (blue) and match (green):

<img src="img\OrderGeometry1.jpeg" width = 300>

The offset and match trades are initally held in a separate **cache** order book and are promoted when the trade is matched. The cache order book is a place for signed but unused trades and has no implact on collateral calculations.

With a small loss of generality, the mechanics of orders here are all unit quantity and there are no partial fills.


#### Geometry of signature chain 

Each trade is chained to a previous trade according to a rule. The order book will require a valid signature (valid signature for message and valid previous node) and a collateral check to ensure the trader has sufficient collateral. Primary orders are chained to a previous valid order, offset are chained to the primary trade.

The figure below is an order to buy at 0.5 with associated offset and match trades.

<img src="img\BasicSignature1.jpeg" width = 300>

If the order is matched by a sell contract at 0.5 the the offset and match contracts are removed from the cache and added to the order book where they remain forever.


##### Example sequence of orders: Single order perfect match

The simplest case is a perfect match of primary trades. Trader 1 enters the market and posts a bid for 1 contracts at 0.5 into the order book, and a corresponding offset and match order into the cache. Offsets and matches share the same root id.
 
*Trader 1:*  
- 1.1 Primary: traderId=1, tradeRootId = 1, tradeBranchId = 1, p=0.5, q=1, isMatched=0  (primary)
- 1.2 Offset: traderId=1, tradeRootId = 1, tradeBranchId = 2, p=0.5, q=-1, isMatched=0  (cache)
- 1.3 Match: traderId=1, tradeRootId = 1, tradeBranchId = 3, p=0.1, q=1, isMatched=1    (cache)


Now trader 2 posts a matching bid for -1 contracts at 0.5, with corresponding offset and match in cache:

*Trader 2:*  
- 2.1 Primary: traderId=2, tradeRootId = 2, tradeBranchId = 1, p=0.5, q=-1, isMatched=0  (primary)
- 2.2 Offset: traderId=2, tradeRootId = 2, tradeBranchId = 2, p=0.5, q=1, isMatched=0  (cache)
- 2.3 Match: traderId=2, tradeRootId = 2, tradeBranchId = 3, p=0.5, q=-1, isMatched=1  (cache)

<img src="img\PerfectMatch.jpeg" width = 600>

Matching proceeds by adding the the offset and match trades to the order book. 

##### Example sequence of orders: Better price match

Now consider a case where trader 1 has a bid at 0.5 and trader 2 offers at  0.4. The trade still matches at 0.5 since trader 1 was there first. Trader 2's offset for the p=0.4 trade is promoted to the order book.


<img src="img\BetterPriceMatch.jpeg" width = 600>

##### Example sequence of orders: Chaining to a previous valid order

Picking up from the previous example now suppose trader 3 enters the market with an offer for 0.7 and a better price offer at 0.8. The order is chained to the highest number trade existing in the order book which is (2.3 from the p=0.5 trade). If another trade came into the order book it would be chained to 3.1 since it is the largest number in the order book.

<img src="img\BetterPriceMatchChained.jpeg" width = 600>


$x^2$

In [2]:

1+1

2

In [4]:
a=1
a

1